#Install Packages

In [1]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 45.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 50.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:

%%capture
!pip install openai
!pip install pandas
!pip install -qqq langchain==0.0.299 --progress-bar off

In [3]:
#!apt-get update
#!apt-get install -y nvidia-driver

In [4]:
!pip install bitsandbytes

In [5]:
from transformers import LlamaTokenizer, LlamaForCausalLM, TrainingArguments, Trainer
#from bitsandbytes.optim import AdamW8bit, LoraConfig, LoRALMHead

#Import Libraries

In [6]:
import datasets
import transformers
import bitsandbytes as bnb
import os
import torch
from transformers import LlamaTokenizer, LlamaForCausalLM, TrainingArguments, Trainer
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
import openai
#import json
#from dotenv import load_dotenv

#Set Model & Parameters

1. Load a llama-2-7b-chat-hf model (chat model)
2. Train it on the mlabonne/guanaco-llama2-1k (1,000 samples), which will produce our fine-tuned model Llama-2-7b-chat-finetune

QLoRA will use a rank of 64 with a scaling parameter of 16. We’ll load the Llama 2 model directly in 4-bit precision using the NF4 type and train it for one epoch

In [7]:
#checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

model_name = "NouranG/Llama-2-7b-humaneval-finetune"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs   can be changed
num_train_epochs = 2

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 50

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

# zyda


#Load everything

1. First of all, we want to load the dataset we defined. Here, our dataset is already preprocessed but, usually, this is where you would reformat the prompt, filter out bad text, combine multiple datasets, etc.


2. Then, we’re configuring bitsandbytes for 4-bit quantization.


3. Next, we're loading the Llama 2 model in 4-bit precision on a GPU with the corresponding tokenizer.


4. Finally, we're loading configurations for QLoRA, regular training parameters, and passing everything to the SFTTrainer. The training can finally start!

In [8]:
import torch



# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)



# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/434 [00:00<?, ?B/s]

In [9]:
def get_user_input_from_file(file_path):
    try:
        with open(file_path, 'r') as f:
            content = f.read()
        return content
    except FileNotFoundError:
        print("File not found.")
        return None

# Example usage:
file_path = '/content/functions_part_4.py'  # Replace with the actual file path
user_input = get_user_input_from_file(file_path)

if user_input is not None:
    print("Content of the file:")
    print(user_input)

Content of the file:
def square_value(num):
    """This function returns the square
    value of the entered number"""
    return num**2


print(square_value(2))
print(square_value(-4))
-------------------------------------------------------------
# Function 8  Fibonacci 

# Function to produce infinite Fibonacci numbers


def function1():
    print("Value in 1st function :", global_var)



def image_convolution(image, kernel):
    """
    Applies a convolution operation to an image using a given kernel.

    Args:
        image (numpy.ndarray): The input image as a 2D array.
        kernel (numpy.ndarray): The convolution kernel as a 2D array.

    Returns:
        numpy.ndarray: The convolved image.
    """
    return convolve(image, kernel)

# Example usage
image = np.array([
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9]
])
kernel = np.array([
    [1, 0, -1],
    [0, 0, 0],
    [-1, 0, 1]
])
print(image_convolution(image, kernel))
------------------------------------------------------

In [10]:
def get_user_input():
    function_code = input("Enter the code for your Python function:\n")
    return function_code

In [11]:
def code_completion(messages, max_tokens=512, model=model):
  prompt = f"complete the code {messages} "
  pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=500)
  result = pipe(f"<s>[INST] {prompt} [/INST]")
  return result[0]['generated_text']

In [12]:
"""def generate_test(completed_code, num_tests=2, model=model, tokenizer=tokenizer):
    functions = [block.strip() for block in completed_code.split("def") if block.strip()]
    test_cases = ""

    for i, function_code in enumerate(functions, start=1):
        prompt = f"Generate at least {num_tests} test cases for the following Python function:\n{function_code}\n then define define if each test pass or fail by putting the test in assert function and explain every test case"
        pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=1000)
        result = pipe(f"<s>[INST] {prompt} [/INST]")
        generated_text = result[0]['generated_text']
        test_cases += f"\nGenerated test cases for Function {i}:\n{generated_text}\n"
    return test_cases"""

'def generate_test(completed_code, num_tests=2, model=model, tokenizer=tokenizer):\n    functions = [block.strip() for block in completed_code.split("def") if block.strip()]\n    test_cases = ""\n\n    for i, function_code in enumerate(functions, start=1):\n        prompt = f"Generate at least {num_tests} test cases for the following Python function:\n{function_code}\n then define define if each test pass or fail by putting the test in assert function and explain every test case"\n        pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=1000)\n        result = pipe(f"<s>[INST] {prompt} [/INST]")\n        generated_text = result[0][\'generated_text\']\n        test_cases += f"\nGenerated test cases for Function {i}:\n{generated_text}\n"\n    return test_cases'

In [13]:
# Prompt the user to specify the number of test cases.
num_tests = int(input("Specify the number of test cases: "))

Specify the number of test cases: 6


In [14]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )


In [15]:
#Another prompt
def generate_test(completed_code, num_tests=2, model=model, tokenizer=tokenizer):
    functions = [block.strip() for block in completed_code.split("def") if block.strip()]
    test_cases = ""

    for i, function_code in enumerate(functions, start=1):
        prompt = f"""Your task is to generate unit test cases for the following function code: {function_code} and doing the following:
                 1. Explain the purpose of the function briefly.
                 2. Write unit tests to cover various scenarios, including edge cases.
                 3. Include assertions to verify the correctness of the function.
                 4. Handle any potential errors or exceptions gracefully.
"""
        pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=1000)
        result = pipe(f"<s>[INST] {prompt} [/INST]")
        generated_text = result[0]['generated_text']
        test_cases += f"\nGenerated test cases for Function {i}:\n{generated_text}\n"
    return test_cases

In [17]:
# Function to extract test functions from generated text by using regular expression
import re
def extract_functions(result):
    test_pattern = r'def test_[^\(]+\(\):(?:\s+.*?)+?assert\s+.+?\n'
    test_functions = re.findall(test_pattern, result, re.DOTALL)
    return test_functions

# Function to count passed and failed test cases
def count_passed_failed(test_functions):
    passed_count = 0
    failed_count = 0
    executed_tests = set()

    for test_function in test_functions:
        function_name = test_function.split('\n')[0].split(' ')[1]
        if function_name in executed_tests:
            continue

        temp_module = {}
        try:
            exec(test_function, temp_module)
            passed_count += 1
        except AssertionError:
            failed_count += 1
        except SyntaxError as e:
            print(f"Syntax error in test function:\n{test_function}\nError: {e}")
            failed_count += 1
        except Exception as e:
            print(f"Error executing test function:\n{test_function}\nError: {e}")
            failed_count += 1

        executed_tests.add(function_name)

    return passed_count, failed_count


In [18]:
# Main execution
if __name__ == "__main__":
    file_path = '/content/functions_part_4.py'  # Replace with the actual file path
    user_input = get_user_input_from_file(file_path)

    if user_input is not None:
        test_cases = generate_test(user_input)
        print("\nGenerated Test Cases:")
        print(test_cases)

        test_functions_text = extract_functions(test_cases)
        passed_count, failed_count = count_passed_failed(test_functions_text)
        print(f"Passed: {passed_count}")
        print(f"Failed: {failed_count}")


Generated Test Cases:

Generated test cases for Function 1:
<s>[INST] Your task is to generate unit test cases for the following function code: square_value(num):
    """This function returns the square
    value of the entered number"""
    return num**2


print(square_value(2))
print(square_value(-4))
-------------------------------------------------------------
# Function 8  Fibonacci 

# Function to produce infinite Fibonacci numbers and doing the following:
                 1. Explain the purpose of the function briefly.
                 2. Write unit tests to cover various scenarios, including edge cases.
                 3. Include assertions to verify the correctness of the function.
                 4. Handle any potential errors or exceptions gracefully.
 [/INST]  Sure, here are some unit test cases for the `square_value` function:
```
def test_square_value_positive_integer(self):
    self.assertEqual(square_value(5), 25)

def test_square_value_negative_integer(self):
    se

In [ ]:
import re

def extract_functions(result):
    # Define a regular expression pattern to match test functions
    test_pattern = r'def test_[^\(]+\(\):(?:\s+.*?)+?assert\s+.+?\n'

    # Extract test functions from the result
    test_functions = re.findall(test_pattern, result, re.DOTALL)

    return test_functions

def count_passed_failed(test_functions):
    # Initialize counters for passed and failed tests
    passed_count = 0
    failed_count = 0

    # Set to store names of executed test functions
    executed_tests = set()

    # Execute each test function and track the results
    for test_function in test_functions:
        # Extract the name of the test function
        function_name = test_function.split('\n')[0].split(' ')[1]

        # Skip execution if the test function has already been executed
        if function_name in executed_tests:
            continue

        # Create a temporary module to execute the test function
        temp_module = {}
        try:
            exec(test_function, temp_module)
            passed_count += 1
        except AssertionError:
            failed_count += 1

        # Add the executed test function to the set
        executed_tests.add(function_name)

    return passed_count, failed_count

In [ ]:
# Example usage:

file_path = '/content/functions_part_4.py'  # Replace with the actual file path
user_input = get_user_input_from_file(file_path)

#user_entered_code = get_user_input()
#completed_code = code_completion(user_input)
test_cases = generate_test(user_input)

#print("\nCompleted Function:")
#print(completed_code)
print("\nGenerated Test Cases:")
print(test_cases)



In [ ]:
if __name__ == "__main__":
    # Example usage:
    test_functions = extract_functions(test_cases)
    passed_count, failed_count = count_passed_failed(test_functions)

    print("Passed:", passed_count)
    print("Failed:", failed_count)

## prompt

In [ ]:
from langchain import PromptTemplate

template = """
<s>[INST] <<SYS>>
Act as an exprt in unit testing who generate test cases for functions.
<</SYS>>

{text} [/INST]
"""

prompt = PromptTemplate(
    input_variables=["text"],
    template=template,
)

In [ ]:
code="""def add(x,y):
  return x+y"""
text = f" \n{code}\n"

In [ ]:
from langchain.chains import LLMChain

chain = LLMChain(llm=model, prompt=prompt)

In [ ]:
template = "<s>[INST] write comprehensive unit tests for the function [function] using assert function . Ensure to cover various scenarios including edge cases and expected behavior. [/INST]"

examples_prompt = PromptTemplate(
    input_variables=["function"],
    template=template,
)
examples_chain = LLMChain(llm=model, prompt=examples_prompt)

In [ ]:
template = "<s>[INST] explain each test case [tests] [/INST]"

examples_prompt = PromptTemplate(
    input_variables=["tests"],
    template=template,
)
examples_chain = LLMChain(llm=llm, prompt=examples_prompt)

In [ ]:
from langchain import PromptTemplate, LLMChain

# Define the first PromptTemplate
template1 = """
<s>[INST] <<SYS>>
Act as an expert in unit testing who generates test cases for functions.
<</SYS>>

{text} [/INST]
"""

prompt1 = PromptTemplate(
    input_variables=["text"],
    template=template1,
)

# Define the second PromptTemplate for writing unit tests
template2 = "<s>[INST] Write comprehensive unit tests for the function [function] using assert function. Ensure to cover various scenarios including edge cases and expected behavior. [/INST]"

prompt2 = PromptTemplate(
    input_variables=["function"],
    template=template2,
)

# Define the third PromptTemplate for explaining each test case
template3 = "<s>[INST] Explain each test case [tests] [/INST]"

prompt3 = PromptTemplate(
    input_variables=["tests"],
    template=template3,
)

# Now, you can create LLMChain objects for interaction
# Make sure you have imported LLMChain from langchain

# examples_chain1 = LLMChain(llm=llm, prompt=prompt1)
# examples_chain2 = LLMChain(llm=llm, prompt=prompt2)
# examples_chain3 = LLMChain(llm=llm, prompt=prompt3)

# Assuming `llm` is the pre-loaded language model object
